## Training an image classifier from scratch on the Cats vs Dogs dataset.

## Introduction

This example shows how to do image classification from scratch, starting from JPEG
image files on disk, without leveraging pre-trained weights or a pre-made Keras
Application model. We demonstrate the workflow on the Kaggle Cats vs Dogs binary
 classification dataset.

We use the `image_dataset_from_directory` utility to generate the datasets, and
we use Keras image preprocessing layers for image standardization and data augmentation.


## Setup


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator

## Load the data: the Cats vs Dogs dataset

### Raw data download

First, let's download the 786M ZIP archive of the raw data:


In [ ]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip


In [ ]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

Now we have a `PetImages` folder which contain two subfolders, `Cat` and `Dog`. Each
 subfolder contains image files for each category.


### Filter out corrupted images

When working with lots of real-world image data, corrupted images are a common
occurence. Let's filter out badly-encoded images that do not feature the string "JFIF"
 in their header.


In [ ]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)


In [ ]:
# load the image
import matplotlib.pyplot as plt

for h in range(10000,10004,1):
  img = load_img('PetImages/Cat/'+str(h)+'.jpg')
  plt.imshow(img)
  plt.show()


## Using image data augmentation

When you don't have a large image dataset, it's a good practice to artificially
introduce sample diversity by applying random yet realistic transformations to the
training images, such as random horizontal flipping or small random rotations. This
helps expose the model to different aspects of the training data while slowing down
 overfitting.


## Generate the Train and the Validation datasets
## with data augmentation


In [ ]:
image_size = (180, 180)

datagen = ImageDataGenerator( rescale=1./255,
        rotation_range=3,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.01, # Randomly zoom image
        width_shift_range=0.01,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.01,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        validation_split=0.2)  # validation split
train_ds = datagen.flow_from_directory(
        "PetImages", target_size=image_size, color_mode='rgb', class_mode='categorical',
         batch_size=64, shuffle=True, seed=13,
        subset='training')
val_ds = datagen.flow_from_directory(
        "PetImages", target_size=image_size, color_mode='rgb', class_mode='categorical',
         batch_size=64, seed=13,
        subset='validation')

## Visualize the data

Here are the first 16 images in the training dataset. As you can see, label 1 is "dog" and label 0 is "cat".


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(11, 11))
images=train_ds[0][0]
labels=train_ds[0][1]
for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].astype("float32"))
#        print(i,labels[i][1])
        plt.title(labels[i][1])
        plt.axis("off")


## Build a model

We'll build a small version of the Xception network. We haven't particularly tried to
optimize the architecture; if you want to do a systematic search for the best model
 configuration, consider using
[Keras Tuner](https://github.com/keras-team/keras-tuner).


In [ ]:
from tensorflow.keras.layers import InputLayer, Input, BatchNormalization
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

inp = Input((180,180,3))
x = Conv2D(32,(5,5),activation='relu',input_shape=(180,180,3))(inp)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)
x = Conv2D(32,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)
x = Conv2D(16,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x= Dense(128,activation='relu')(x)
x= BatchNormalization()(x)
x = Dropout(0.3)(x)
out = Dense(2,activation='softmax')(x)
model = Model(inp,out)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
plot_model(model, show_shapes=True)

## Train the model


In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

### We can obtain to >90% validation accuracy


## Run inference on new data

Note that data augmentation and dropout are inactive at inference time.


In [ ]:
import numpy as np
img = load_img('cane.jpg', target_size=(180,180))
A = keras.preprocessing.image.img_to_array(img)/255.
B = A.reshape(1,180,180,3)
my_prediction = model.predict(B)
np.set_printoptions(suppress=True,precision=4)
print("Your algorithm predicts",  my_prediction)
